# Bundle Resources
- Access Postgres DB
- Grab individual patients and their related resources
- Check size of bundles (before sending off)

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import json
import psycopg2
import requests
import matplotlib

In [ ]:
# create a database connection
sqluser = 'postgres'
sqlpass = 'postgres'
dbname = 'mimic'
host = 'localhost'

con = psycopg2.connect(dbname=dbname, user=sqluser, password=sqlpass, host=host)
cur = con.cursor()

## Grab individual patient

In [ ]:
q_patient = "SELECT * FROM mimic_fhir.patient LIMIT 10;"
patient1 = pd.read_sql_query(q_patient,con)

In [ ]:
patient1.id[1]

## Grab patient plus encounter/condition

In [ ]:
q_encounter = f"""SELECT * 
                FROM mimic_fhir.encounter 
                WHERE patient_id = '{patient1.id[1]}';"""
q_encounter
encounters = pd.read_sql_query(q_encounter,con)

In [ ]:
encounters

In [ ]:
q_condition = f"""SELECT * 
                FROM mimic_fhir.condition 
                WHERE patient_id = '{patient1.id[1]}';"""
q_condition
conditions = pd.read_sql_query(q_condition,con)

In [ ]:
conditions

## Bundle basic resources for patient
- patient
- encoutner
- condition

In [ ]:
class FhirBundle:
    def __init__(self,id):
        self.resourceType = 'Bundle'
        self.type = 'transaction'
        self.id = id
        self.entry = []
        
    def add_entry(self, resource, request):        
        new_request = {}
        new_request['method'] = request
        if request == 'POST':
            new_request['url'] = resource['resourceType']
        elif request == 'PUT':
            new_request['url'] = resource['resourceType'] +'/' + resource['id']
        else:
            raise Exception(f'Request {request}, is not currently supported')
        
        new_entry = {}
        new_entry['resource'] = resource
        new_entry['request'] = new_request   
        new_entry['fullUrl'] = resource['id']
        self.entry.append(new_entry)
    
    def to_json(self):
        return self.__dict__
    
    def get_size(self):
        return len(self.entry)
    
    def request(self):
        url = 'http://localhost:8080/fhir/'
        #requests.post(url,  json = self.to_json(), headers={"Content-Type": "application/fhir+json"} )
        resp = requests.post(url,  json = self.to_json(), headers={"Content-Type": "application/fhir+json"} )
        output = json.loads(resp.text)
        return output

In [ ]:
b_pat.to_json()['entry'][40]

In [ ]:
b_pat.to_json()['entry'][41]

In [ ]:
%%time
#table_names = ['encounter', 'condition', 'procedure'] #, 'observation_labs'
table_names = ['observation_labs']

q_patient = "SELECT * FROM mimic_fhir.patient;"
patients = pd.read_sql_query(q_patient,con)
for index, pat in patients.iterrows():
    b_pat = FhirBundle('patient-bundle')
    b_pat.add_entry(pat.fhir, 'PUT')
    for table in table_names:
        q_table = f"""SELECT * 
                    FROM mimic_fhir.{table} 
                    WHERE patient_id = '{pat.id}';"""
        resources = pd.read_sql_query(q_table,con)
        for _, resource in resources.iterrows():
            b_pat.add_entry(resource.fhir, 'PUT')
    # send patient bundle!
    # check size for now
    #print(f'Patient {pat.id} size is: {b_pat.get_size()}')
    if index %50 == 0:
        print(f'Patient {index}')
    if b_pat.get_size() < 100:
        output = b_pat.request()
        if output['resourceType'] == 'OperationOutcome':
            print(output['issue'])
            break
    else:
        print(f'Patient {pat.id} size is: {b_pat.get_size()}')
        
    
print('-----------------------------------')
print('COMPLETE')  

# Testing 123
            
# send full microbio bundle        
#output = b_micro.request()



## Validate individual resource

In [ ]:
%%time
server = 'http://localhost:8080/fhir/'


table = 'observation_micro_org'
q_table = f"""SELECT * 
            FROM mimic_fhir.{table} 
            LIMIT 3; """
resources = pd.read_sql_query(q_table,con)
for _, resource in resources.iterrows():

    url = server +  resource.fhir['resourceType'] + '/' + resource.fhir['id']
    resp = requests.put(url,  json = resource.fhir, headers={"Content-Type": "application/fhir+json"} )
    output = json.loads(resp.text)
  
    